In [ ]:
## Task 1. Install packages and import libraries
!pip install asyncio==3.4.3 asyncpg==0.27.0 cloud-sql-python-connector["asyncpg"]==1.2.3
!pip install numpy==1.22.4 pandas==1.5.3
!pip install pgvector==0.1.8
!pip install langchain==0.0.196 transformers==4.30.1
!pip install google-cloud-aiplatform==1.26.0

In [ ]:
### Install Vertex AI SDK & Other dependencies

!sudo apt -y -qq install tesseract-ocr
!sudo apt -y -qq install libtesseract-dev
!sudo apt-get -y -qq install poppler-utils #required by PyPDF2 for page count and other pdf utilities
!sudo apt-get -y -qq install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig
! pip install --upgrade google-cloud-aiplatform
# LangChain
! pip install langchain langchain-experimental langchain[docarray]
! pip install pypdf
! pip install shapely==2.0.0
! pip install pydantic==1.10.8
# Open source vector store
! pip install chromadb==0.3.26
! pip install typing-inspect==0.8.0 typing_extensions==4.5.0
# For dense vector representations of text
! pip install sentence-transformers
! pip install langchain-google-vertexai
! pip install -qU langchain-google-vertexai


In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()

app.kernel.do_shutdown(True)

In [ ]:
# Import libraries

import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain_google_vertexai import ChatVertexAI

warnings.filterwarnings("ignore")

In [ ]:
## Task 2. Load the model and prepare data
from langchain_google_vertexai import ChatVertexAI

# Load the pre-trained model
model = ChatVertexAI(model_name="gemini-pro")
#TODO: Load the pre-trained text generation model named gemini-pro using ChatVertexAI.
#TODO: Load the pre-trained text generation model named Quicklab using ChatVertexAI.
import urllib.request
from pathlib import Path

# Define the URL and local path
url = "https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf"
local_path = Path("data/practitioners_guide_to_mlops_whitepaper.pdf")

# Create the 'data' directory if it doesn't exist
local_path.parent.mkdir(parents=True, exist_ok=True)

# Download and save the PDF file
urllib.request.urlretrieve(url, local_path)

#TODO: Download a PDF file from specified URL and save it in "data" directory
from langchain.document_loaders import PyPDFLoader

# Load the PDF file
pdf_loader = PyPDFLoader(str(local_path))

# Load and split the PDF into individual pages
pages = pdf_loader.load_and_split()


In [ ]:
## Task 3. Generate summaries

In [ ]:
#Prompt design with Stuffing Chain

import os
import sys

import vertexai
from langchain.chains import RetrievalQA
from langchain.llms.vertexai import VertexAI
from langchain.prompts import PromptTemplate

from langchain import PromptTemplate
from langchain_google_vertexai import ChatVertexAI
from langchain.chains import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
import urllib.request
from pathlib import Path

# Define the URL and local path
url = "https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf"
local_path = Path("data/practitioners_guide_to_mlops_whitepaper.pdf")

# Create the 'data' directory if it doesn't exist
local_path.parent.mkdir(parents=True, exist_ok=True)

# Download and save the PDF file
urllib.request.urlretrieve(url, local_path)

# Load the PDF file
pdf_loader = PyPDFLoader(str(local_path))

# Load and split the PDF into individual pages
pages = pdf_loader.load_and_split()

# Prepare the prompt template
prompt_template = """Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
  """

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

# Initialize the pre-trained model
model = ChatVertexAI(model_name="gemini-pro")

# Load the summarization chain with the stuff method
summarization_chain = load_summarize_chain(
    llm=model,
    chain_type="stuff",
    prompt=prompt
)

# Prepare the first three pages as documents
input_documents = pages[:3]  # Use first three Document objects

# Generate the summary
summary = summarization_chain({"input_documents": input_documents})

print(summary)


In [ ]:
## Task 4. Add tools to the LLM for multiplication

In [ ]:
from langchain_core.tools import tool
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-pro")
import getpass
import os

In [ ]:
## Subscribe To Quicklab :-0

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]

In [ ]:
#TODO: Add the tools to the LLM created earlier and invoke it with the following query. Print the result in the console.
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
query = "What is 3 * 12?"
messages = [HumanMessage(query)]


In [ ]:
#TODO: Iterate through the tools in the response, invoke the tools and append the response to the messages object.
llm_with_tools = llm.bind_tools(tools)
always_multiply_llm = llm.bind_tools([multiply], tool_choice="multiply")
always_call_tool_llm = llm.bind_tools([add, multiply], tool_choice="any")
query = "What is 3 * 12?"

llm_with_tools.invoke(query).tool_calls



In [ ]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.messages import HumanMessage, ToolMessage


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

In [ ]:
llm_with_tools.invoke(messages)